# Visualización dinámica

En esta sesión estudiaremos algunas técnicas para la visualización dinámica de conjuntos de datos que tienen alguna componente temporal, haciendo especial énfasis en aquellos problemas para los que el dinamismo en la representación realmente supone alguna ventaja con respecto a visualizaciones estáticas clásicas. 

Comenzaremos con una breve descripción técnica de las herramientas que utilizaremos para generar las animaciones, así como su integración con `notebook`. A continuación, presentaremos diferentes ejemplos de problemas para los cuales la visualización dinámica permite comprender más en detalle el conjunto de datos o su comportamiento.

## Visualización interactiva con <tt>matplotlib</tt>

Matplotlib da soporte explícito para la creación de animaciones a través de la API [`matplotlib.animation`](http://matplotlib.org/api/animation_api.html). En este paquete podemos encontrar distintas clases y métodos para crear animaciones, controlar su ciclo de vida, reproducirlas y exportarlas a un formato conveniente. En la documentación oficial podemos encontrar también diversos [ejemplos ilustrativos](http://matplotlib.org/1.5.1/examples/animation/).

Esta API está soportada en `notebook` [a partir de la versión 1.4 de `matplotlib`](http://matplotlib.org/users/whats_new.html#new-in-matplotlib-1-4) mediante el uso de un backend interactivo.Esto es necesario debido a que la opción clásica `%matplotlib inline` sólo permite generar imágenes estáticas. Este backend se denomina `notebook`, y lo seleccionaremos con el siguiente [`magic`](http://ipython.readthedocs.org/en/stable/interactive/tutorial.html#magic-functions):

In [ ]:
%matplotlib notebook

Comprobamos las capacidades de este backend creando una gráfica sencilla:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('ggplot')
plt.plot(np.arange(-5,5,0.1)**3);

Como podemos ver, la gráfica resultante es interactiva, lo que nos permite hacer zoom, desplazar los ejes, modificar el tamaño, etc. Si en algún momento deseamos terminar esta interactividad, podemos hacer que la gráfica se convierta en una imagen estática, equivalente a si usásemos el backend `inline`.

## Análisis de información sociológica

En esta sección utilizaremos animaciones para analizar distintos tipos de información sociológica. El sitio web [Gapminder](http://www.gapminder.org/) es un magnífico repositorio de este tipo de información a nivel mundial, incluyendo series históricas desde el año 1800. En la hoja de cálculo <tt>social_data.xlsx</tt> se incluye información descargada de esta web para distintas variables y países.

Nuestro primer objetivo será *visualizar la evolución temporal de la esperanza de vida de los habitantes de un conjunto de países frente a la [renta per cápita, o producto interior bruto (GDP) por habitante](https://en.wikipedia.org/wiki/Per_capita_income)*.

Hacemos una breve exploración inicial de nuestros datos:

In [ ]:
import pandas as pd
from IPython.display import display, HTML
df = pd.read_excel('social_data.xlsx', sheet_name=None)
#Para cada hoja de cálculo dentro del archivo, mostramos la cabecera del DataFrame como HTML.
for sh in df:
    display(HTML('<h3>' + sh + '</h3>' + df[sh].head().to_html()))

Como podemos ver, disponemos de información sobre la renta per cápita (GDP), la esperanza de vida, y la población de cada país desde el año 1800. 

En la primera parte de la práctica, analizaremos la relación entre esperanza de vida y renta per cápita de los siguientes países: China, EEUU, Brasil, Alemania, Sudáfrica y Australia.

In [ ]:
countries = ['China', 'United States', 'Brazil', 'Germany', 'South Africa', 'Australia']
#Indexamos los Dataframes de "GDP" y "Life Expectancy" por países, y filtramos los seleccionados.
gdp = df['GDP'].set_index('Country').loc[countries]
lfexp = df['Life Expectancy'].set_index('Country').loc[countries]
display(gdp)

Como primera exploración visual, dibujamos un gráfico de burbujas que muestre la relación en el año 2015 entre la esperanza de vida y la renta per cápita para los países seleccionados.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
#Configuración de las etiquetas de los ejes
plt.xlabel('Income per person (Inflation-adjusted $)')
plt.ylabel('Life Expectancy (years)')
#Creación de un punto para cada país
for country in countries:
    plt.plot([gdp[2015][country]], [lfexp[2015][country]], 'o', label=country, markersize=30)
#Leyenda:
ax.legend(loc='lower right', fontsize=12, markerscale=0.5, numpoints=1, frameon=False)

<div style="font-size:125%; display:inline; font-weight:bold">Ejercicio:</div> *Crear una animación a partir de la gráfica anterior, que permita visualizar la evolución de la relación entre renta per cápita y esperanza de vida durante toda la serie temporal disponible.* 

La forma más simple y recomendada de crear una animación en matplotlib es a través de la función [`FuncAnimation`](http://matplotlib.org/api/animation_api.html#matplotlib.animation.FuncAnimation), que genera el conjunto de *frames* a partir de la ejecución repetida de una función definida por el usuario (en este caso, la función `update_plot`). Los valores de los argumentos pasados a esta función y el intervalo entre imágenes consecutivas se controlan con los parámetros `frames` e `interval`.

In [ ]:
import matplotlib.animation as animation
fig = plt.figure()
ax = fig.add_subplot(111)
#Configuración de los límites de la gráfica y de las etiquetas de los ejes.
ax.set_xlim((0, 60000))
ax.set_ylim((0, 90))
plt.xlabel('Income per person (Inflation-adjusted $)')
plt.ylabel('Life Expectancy (years)')
#Creación de un punto para cada país, inicialmente sin datos.
circs = {}
for country in countries:
    circs[country] = plt.plot([], [], 'o', label=country, markersize=30)[0]
#Leyenda y etiqueta para el año actual.
ax.legend(loc='lower right', fontsize=12, markerscale=0.5, numpoints=1, frameon=False)
cur_year = ax.annotate("", (0.05, 0.9), xycoords='axes fraction', size='x-large', weight='bold')

def update_plot(year):
    """
    Función de animación, en la que se actualiza la posición del punto correspondiente
    a cada país, así como la etiqueta del año actual.
    """
    #Actualizar el centro de cada punto con set_data
    for country in countries:
        circs[country].set_data([gdp[year][country]], [lfexp[year][country]])
    #Actualizar la etiqueta del año correspondiente con set_text
    cur_year.set_text(str(year))
    return list(circs.values()) + [cur_year]

#Creamos la animación, ejecutando la función 'update_plot' para cada año del dataframe
ani = animation.FuncAnimation(fig, update_plot, frames=gdp.keys(), interval=100, repeat=False)

### Exportando animaciones a través del paquete `JSAnimation`

Otro método recomendado para visualizar animaciones en un `notebook` es el uso de la librería [JSAnimation](https://github.com/jakevdp/JSAnimation). La [instalación es muy sencilla](https://gist.github.com/gforsyth/188c32b6efe834337d8a), e incluso podemos descargar directamente las fuentes del proyecto e importarlas desde nuestro código, al tratarse de un proyecto muy pequeño. Para instalar `JSAnimation` en [Anaconda](https://www.continuum.io/downloads), podemos utilizar el siguiente comando:

```
conda install -c conda-forge jsanimation
```

Esta librería evita que sea necesario utilizar un intérprete de Python cada vez que deseemos reproducir una animación ya generada, pues esta se almacena completamente en el código HTML y se reproduce utilizando Javascript.

Antes de utilizar `JSAnimation`, se recomienda seleccionar el backend `inline` de matplotlib. A continuación, simplemente debemos importar la función `display_animation`.

In [ ]:
%matplotlib inline
from matplotlib import animation
from IPython.display import HTML

Esta función recibe cualquier objeto de tipo [`Animation`](http://matplotlib.org/api/animation_api.html#matplotlib.animation.Animation) y genera un HTML dinámico que puede incluirse en cualquier página web, incluido un `notebook` exportado como HTML. Además, se incluye un conjunto de controles que nos permiten pausar, modificar la velocidad de la animación, o seleccionar un instante concreto de la misma. El único inconveniente de usar este paquete es que la animación debe ser renderizada completamente antes de comenzar a reproducirse, lo que resulta incómodo para programar y depurar. Además, el archivo de `notebook` resultante de usar este paquete será considerablemente más grande, pues almacena la información binaria de todas las imágenes que componen las animaciones.

A continuación se muestra la misma animación del ejemplo anterior renderizada a través de `JSAnimation`.

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
#Volvemos a seleccionar el backend notebook para las siguientes animaciones
%matplotlib notebook

<div style="font-size:125%; display:inline; font-weight:bold">Ejercicio:</div> *Crear una nueva animación en la cual el tamaño del marcador correspondiente a cada país se actualice en función de la población. Utilizar para ello la función `get_population_markersize` que se proporciona a continuación:*

In [ ]:
def get_population_markersize(pop):
    """
    Función para obtener el tamaño del marcador correspondiente a una determinada población.
    A una población de 2 millones se corresponde un tamaño 4.0, que escalará de forma que
    la superficie del marcador sea proporcional a esta relación.
    """
    return 4*np.sqrt(pop/2e6)

Además de esta función, es necesario tener en cuenta que los datos de población no están disponibles para todos los años en la serie temporal.

In [ ]:
pop = df['Population'].set_index('Country').loc[countries]
display(pop)

In [ ]:
#Actualizamos el conjunto de años para los DataFrames de "GDP" y "Life Expectancy"
#Nota: pop.keys() puede resultar de ayuda.
gdp_filt = gdp[pop.keys()[:-10]]
lfexp_filt = lfexp[pop.keys()[:-10]]
display(gdp_filt)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
#Configuración de los límites de la gráfica y de las etiquetas de los ejes.
ax.set_xlim((0, 60000))
ax.set_ylim((0, 90))
plt.xlabel('Income per person (Inflation-adjusted $)')
plt.ylabel('Life Expectancy (years)')
#Creación del punto para cada país
circs = {}
for country in countries:
    circs[country] = plt.plot([], [], 'o', label=country, markersize=30)[0]
#Leyenda y etiqueta para el año actual.
ax.legend(loc='lower right', fontsize=12, markerscale=0.5, numpoints=1, frameon=False)
cur_year = ax.annotate("", (0.05, 0.9), xycoords='axes fraction', size='x-large', weight='bold')

def update_plot(year):
    """
    Función de animación, en la que se actualiza la posición y el tamaño del punto
    correspondiente a cada país, así como la etiqueta del año actual.
    """
    #En este caso, además de actualizar la posición de cada punto con "set_data", debemos
    #actualizar el tamaño de cada punto con "set_markersize"
    for country in countries:
        circs[country].set_data([gdp[year][country]], [lfexp[year][country]])
        circs[country].set_markersize(get_population_markersize(pop[year][country]))
    cur_year.set_text(str(year))
    return list(circs.values()) + [cur_year]

#Creamos la animación con "FuncAnimation".
ani = animation.FuncAnimation(fig, update_plot, frames=gdp_filt.keys(), interval=100, repeat=False)

<div style="font-size:125%; display:inline; font-weight:bold">Ejercicio opcional:</div> *Repetir la animación anterior, pero incluyendo todos los años entre 1800 y 2015. Para aquellos años en los cuales no se dispone de datos de población, realizar una interpolación lineal para obtener una estimación del valor.*

<div style="font-size:125%; display:inline; font-weight:bold">Ejercicio:</div> *Crear una animación que muestre las pirámides de población resultantes de las [proyecciones de población para los años 2015-2064 del Instituto Nacional de Estadística](http://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176953&menu=resultados&idp=1254735572981), que se encuentran en el fichero `population_projection.xlsx`*.

En primer lugar, leemos los datos, e indexamos cada una de las tablas (hombres y mujeres) por la edad, para facilitar el acceso a los datos de interés.

In [ ]:
df = pd.read_excel('population_projection.xlsx', sheet_name=None)
men, women = df['Varones'], df['Mujeres']
men.set_index('Edad', inplace=True)
women.set_index('Edad', inplace=True)
display(women.head(), women.tail())

Como vemos, disponemos de datos para cada posible valor de edad entre 1 y 99 años. Sin embargo, a la hora de realizar pirámides de población, es habitual establecer grupos de edad [en márgenes de 5 años](http://www.ine.es/prensa/np813.pdf). Esto lo podemos hacer de forma muy sencilla con pandas y las funciones [`cut()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.cut.html) y [`groupby()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html).

In [ ]:
categories, bins = pd.cut(women.index, np.append(women.index[::5], 100), retbins=True, )
#Tanto para la serie de varones como de mujeres, debemos agrupar por las categorías creadas,
#utilizando la función "groupby", y a continuación agregar el resultado sumando los valores
#de cada categoría, con la función "aggregate".
gwomen = women.groupby(categories).aggregate(np.sum)
gmen = men.groupby(categories).aggregate(np.sum)
gwomen.head()

Para la visualización de los datos, el método [`barh()`](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.barh) de matplotlib nos proporciona una forma directa de representar las pirámides de población, necesitando únicamente pequeños ajustes estéticos.

In [ ]:
#Creamos dos gráficas que comparten el eje Y, y eliminamos el espacio horizontal
fig, (axw,axm)= plt.subplots(ncols=2, sharey=True)
plt.subplots_adjust(wspace=0)
#Primer año en la serie temporal
first_year = next(iter(gwomen))
#Etiqueta que muestra el año que se está visualizando
cur_year = plt.suptitle(str(first_year), size='x-large', weight='bold')
#Obtenemos el máximo de la coordenada X
xmax = max(np.amax(gwomen.values), np.amax(gmen.values))
#Establecemos el rango de X para las dos gráficas
axw.set_xlim((0, 1.1*xmax))
axm.set_xlim((0, 1.1*xmax))
#La gráfica de mujeres (a la izquierda) deberá estar invertida
axw.invert_xaxis()
#Otros ajustes estéticos
axw.yaxis.set_ticks(bins)
axm.tick_params(labelright=True)
axw.set_title('Mujeres')
axm.set_title('Varones')
#Creamos la gráfica para mujeres y para varones, con la función barh
wbars = axw.barh(bins[:-1], gwomen[first_year].values, height=5)
mbars = axm.barh(bins[:-1], gmen[first_year].values, height=5)

def update_pyramid(year):
    """Actualizamos la pirámide de población"""
    #Para cada una de las barras en "wbars" y "mbars", debemos actualizar
    #su tamaño con la función "set_width". Finalmente, actualizaremos
    #también el título, referenciado por la etiqueta "cur_year"
    for age, (wbar, mbar) in enumerate(zip(wbars.patches, mbars.patches)):
        wbar.set_width(gwomen[year][age])
        mbar.set_width(gmen[year][age])
    cur_year.set_text(str(year))
    return (wbars, mbars)

#Creamos la animación, utilizando FuncAnimation y pasando como parámetro "frames" el conjunto
#de años para los que disponemos de datos. En este caso, el parámetro "interval" será de 500ms
ani = animation.FuncAnimation(fig, update_pyramid, frames=iter(gwomen), interval=500, repeat=False)

## Análisis de algoritmos

En esta práctica hemos visto ejemplos de visualizaciones dinámicas que nos ayudan a comprender mejor datos temporales, o a descubrir información adicional que no resulta fácilmente accesible mediante visualizaciones estáticas. Pero además de esto, una visualización dinámica puede resultar de gran ayuda para comprender el funcionamiento de ciertos algoritmos, y en particular aquellos que utilizaremos para construir modelos a partir de conjuntos de datos.

A continuación se muestra un ejemplo, [inspirado en el que podemos encontrar en la documentación de Scikit-learn](http://scikit-learn.org/stable/modules/svm.html), construiremos una animación que permita visualizar el proceso de aprendizaje de un clasificador SVM, lo que nos permitirá comprobar cómo los hiperplanos discriminantes de las distintas clases se van actualizando con la aparición de nuevos ejemplos.

In [ ]:
from sklearn import svm, datasets
import matplotlib.colors

#Conjunto de colores utilizado para el pintado
cols = ['#E24A33', '#348ABD', '#8EBA42']
#Cargamos el conjunto de datos
iris = datasets.load_iris()
X, y = iris.data[:, :2], iris.target
#Para hacer dinámico el aprendizaje, modificaremos los pesos de cada
#ejemplo, incluyendo uno más en cada iteración.
sample_weights = np.zeros_like(y)
#Reordenamiento aleatorio de los ejemplos
transp = np.random.permutation(np.c_[X,y])
X, y = transp[:, :2], transp[:, 2].astype(int)

#Instancia de SVM que utilizaremos para el aprendizaje
vm = svm.SVC()

#Creamos una malla para dibujar el área de cada clase
h = .02
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

fig = plt.figure()
#Configuramos los ejes
ax = fig.add_subplot(111)
ax.set_xlabel('Sepal length')
ax.set_ylabel('Sepal width')
ax.set_xlim(xx.min(), xx.max())
ax.set_ylim(yy.min(), yy.max())
ax.set_xticks(())
ax.set_yticks(())
ax.set_title('SVM learning process')
#Añadimos las gráficas que dibujarán los puntos de cada una de las clases.
pts = {k : ax.plot([], [], 'o', color=cols[k])[0] for k in np.unique(y)}
#Dibujo del área de clasificación de cada clase.
contours = None

def update_classifier(nsamp):
    #Eliminamos los contornos de cada clase
    global contours
    if contours is not None:
        for cont in contours.collections:
            ax.collections.remove(cont)
    #Añadimos un nuevo ejemplo al entrenamiento
    sample_weights[nsamp] = 1
    vm.fit(X, y, sample_weights)
    #Calculamos la salida para cada punto de la malla
    Z = vm.predict(np.c_[xx.ravel(), yy.ravel()])
    #Y la dibujamos
    Z = Z.reshape(xx.shape)
    contours = ax.contourf(xx, yy, Z, alpha=0.8, colors=cols, levels=[0, 0.5, 1.5, 2])
    #Añadimos el nuevo punto a la gráfica correspondiente.
    lastx, lasty = X[nsamp]
    xpt, ypt = pts[y[nsamp]].get_data()
    pts[y[nsamp]].set_data(np.append(xpt, lastx), np.append(ypt, lasty))
    return list(pts.values()) + contours.collections

ani = animation.FuncAnimation(fig, update_classifier, len(y), interval=100, repeat=False)

## Recursos adicionales

En esta introducción a las herramientas de visualización dinámica nos hemos centrado en el paquete `matplotlib`, al ser el más popular dentro del ecosistema Python. Sin embargo, existen otras librerías, normalmente basadas o compatibles con `matplotlib`, que pueden resultar muy útiles a la hora de crear visualizaciones dinámicas e interactivas. Destacamos las siguientes:

 - Bokeh: http://bokeh.pydata.org/en/latest/
 - Seaborn: https://stanford.edu/~mwaskom/software/seaborn/
 - NVD3: https://github.com/areski/python-nvd3
 - MPLD3: http://mpld3.github.io/